In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#import the datasets

train_data = pd.read_csv('data_in/train_data.csv')
test_data = pd.read_csv('data_in/test_data.csv')
print(train_data.head())


         date  product_identifier  department_identifier  \
0  2012-01-01                  74                     11   
1  2012-01-01                 337                     11   
2  2012-01-01                 423                     12   
3  2012-01-01                 432                     12   
4  2012-01-01                 581                     21   

          category_of_product  outlet        state  sales  
0                      others     111  Maharashtra      0  
1                      others     111  Maharashtra      1  
2                      others     111  Maharashtra      0  
3                      others     111  Maharashtra      0  
4  fast_moving_consumer_goods     111  Maharashtra      0  


In [3]:
#Auxilary datasets
product_prices = pd.read_csv('data_in/product_prices.csv')
date_week_map = pd.read_csv('data_in/date_to_week_id_map.csv')

print(product_prices.head())

   outlet  product_identifier  week_id  sell_price
0     111                  74       49        2.94
1     111                  74       50        2.94
2     111                  74       51        2.94
3     111                  74       52        2.94
4     111                  74       53        2.94


In [4]:
#sales is target attribute
#You are expected to create an analytical and forecasting framework to 
# predict the sales of the products based on the quantitative and 
# qualitative features provided in the datasets. 
# You may derive new features from the existing features and
# also from the domain knowledge, which may help in 
# improving the model efficiency

In [5]:
#data cleaning check data sanity
#merge main data and the required auxilary data
#EDA: Simple EDA's understand the category and state split 

In [7]:
print(train_data.shape)
print(product_prices.shape)
print(date_week_map.shape)

(395000, 7)
(59000, 4)
(821, 2)


In [8]:
#Merging the data sets

product_week_map = pd.merge(
    product_prices,
    date_week_map,
    on = ['week_id'],
    how = 'inner'
)

In [11]:
print(product_week_map.head())
print(product_week_map.shape)

   outlet  product_identifier  week_id  sell_price        date
0     111                  74       49        2.94  2012-01-01
1     111                  74       49        2.94  2012-01-02
2     111                  74       49        2.94  2012-01-03
3     111                  74       49        2.94  2012-01-04
4     111                  74       49        2.94  2012-01-05
(410500, 5)


In [12]:
train_data = pd.merge(train_data, product_week_map,
                      on = ['date','product_identifier','outlet'],
                      how = 'inner')

In [13]:
train_data.shape

(395000, 9)

In [14]:
train_data.head()

,date,product_identifier,department_identifier,category_of_product,outlet,state,sales,week_id,sell_price
0,2012-01-01,74,11,others,111,Maharashtra,0,49,2.94
1,2012-01-01,337,11,others,111,Maharashtra,1,49,7.44
2,2012-01-01,423,12,others,111,Maharashtra,0,49,0.97
3,2012-01-01,432,12,others,111,Maharashtra,0,49,4.97
4,2012-01-01,581,21,fast_moving_consumer_goods,111,Maharashtra,0,49,4.88


In [15]:
train_data.describe()

,product_identifier,department_identifier,outlet,sales,week_id,sell_price
count,395000.000000,395000.000000,395000.000000,395000.000000,395000.000000,395000.000000
mean,1509.960000,24.460000,211.200000,1.228919,105.070886,4.987644
std,809.799518,6.337863,91.161291,3.595266,32.578749,3.874444
min,74.000000,11.000000,111.000000,0.000000,49.000000,0.050000
25%,926.000000,21.000000,113.000000,0.000000,77.000000,2.680000
50%,1325.000000,22.000000,221.500000,0.000000,105.000000,3.980000
75%,1753.000000,31.000000,331.000000,1.000000,133.000000,6.480000
max,3021.000000,33.000000,333.000000,293.000000,161.000000,44.360000


In [16]:
#check for null values
train_data.isnull().sum()

date                     0
product_identifier       0
department_identifier    0
category_of_product      0
outlet                   0
state                    0
sales                    0
week_id                  0
sell_price               0
dtype: int64

In [17]:
#we don't have any null values in the dataset 
#lets check first 5 and last 5 rows

In [18]:
train_data.head()


,date,product_identifier,department_identifier,category_of_product,outlet,state,sales,week_id,sell_price
0,2012-01-01,74,11,others,111,Maharashtra,0,49,2.94
1,2012-01-01,337,11,others,111,Maharashtra,1,49,7.44
2,2012-01-01,423,12,others,111,Maharashtra,0,49,0.97
3,2012-01-01,432,12,others,111,Maharashtra,0,49,4.97
4,2012-01-01,581,21,fast_moving_consumer_goods,111,Maharashtra,0,49,4.88


In [19]:
train_data.tail()

,date,product_identifier,department_identifier,category_of_product,outlet,state,sales,week_id,sell_price
394995,2014-02-28,2932,33,drinks_and_food,333,Kerala,2,161,2.78
394996,2014-02-28,2935,33,drinks_and_food,333,Kerala,8,161,0.20
394997,2014-02-28,3004,33,drinks_and_food,333,Kerala,0,161,2.50
394998,2014-02-28,3008,33,drinks_and_food,333,Kerala,0,161,1.98
394999,2014-02-28,3021,33,drinks_and_food,333,Kerala,0,161,2.08


In [21]:
train_data.dtypes

date                      object
product_identifier         int64
department_identifier      int64
category_of_product       object
outlet                     int64
state                     object
sales                      int64
week_id                    int64
sell_price               float64
dtype: object

In [22]:
##converting as category
col = ['category_of_product','state']
train_data[col] = train_data[col].astype('category')

In [ ]:
train_data.dtypes